<a href="https://colab.research.google.com/github/PCiunkiewicz/ensf-619/blob/master/colab/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --quiet -U torch torchmetrics lightning torchvision seaborn nibabel
!git clone https://github.com/PCiunkiewicz/ensf-619.git

In [ ]:
# Optional for nightly release; may need to update cuda version tag
!pip install -U --pre torch torchvision --index-url https://download.pytorch.org/whl/nightly/cu117

In [ ]:
# Optional for TPU support; may need to update tpu-pytorch target url
!pip install cloud-tpu-client https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-2.0-cp39-cp39-linux_x86_64.whl

In [ ]:
!git -C ensf-619/ pull

In [2]:
import os
import sys
sys.path.insert(0, '/content/ensf-619/final_project')

import numpy as np
import matplotlib.pyplot as plt
import torch
from tqdm import tqdm

import torch
from lightning.pytorch import Trainer
from lightning.pytorch.tuner import Tuner
from lightning.pytorch.callbacks import LearningRateMonitor, ModelCheckpoint
from torch.utils.data import random_split, DataLoader
from torchmetrics.functional import structural_similarity_index_measure as ssim, peak_signal_noise_ratio as psnr

from utils import nrmse, nrmse2, custom_imshow
from paths import MODEL_PATH
from data import load_data, DeepCascadeDataset
from model import DeepCascade
from transform import DeepCascadeTransform

NUM_WORKERS = os.cpu_count()

In [3]:
SIZE = 164

images, masks = load_data(size=SIZE, mode='newborn')
images = torch.tensor(images, dtype=torch.float32).to('cuda')
images = images.unsqueeze(1)
masks = torch.tensor(masks, dtype=torch.float32).to('cuda')

transform = DeepCascadeTransform(size=SIZE)
dataset = DeepCascadeDataset(images, masks, transform=transform)
dataset.val = True

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir /content/drive/MyDrive/ENSF619/models/DeepCascade/lightning_logs

In [5]:
model = DeepCascade.load_model('version_10').eval()
model.to('cuda')

def batch_metrics(y_pred, y_true, prefix=''):
    return {
        f'{prefix}ssim:': ssim(y_pred.unsqueeze(1), y_true.unsqueeze(1), data_range=1),
        f'{prefix}psnr:': psnr(y_pred, y_true, data_range=1),
        f'{prefix}nrmse:': nrmse(y_pred, y_true),
        f'{prefix}nrmse2:': nrmse2(y_pred, y_true)
    }

rec = []
imgs = []
pred = []
for kspace, mask, img in tqdm(DataLoader(dataset, batch_size=32)):
    rec.append(torch.abs(torch.fft.ifft2(kspace[:,0] + 1j * kspace[:,1])))
    imgs.append(img)
    pred.append(model(kspace, mask).detach())

rec = torch.cat(rec)
img = torch.cat(imgs)
pred = torch.cat(pred)

rec_metrics = batch_metrics(rec, img, prefix='rec_')
print(rec_metrics)
pred_metrics = batch_metrics(pred, img, prefix='pred_')
print(pred_metrics)

100%|██████████| 82/82 [00:21<00:00,  3.88it/s]


{'rec_ssim:': tensor(0.7177, device='cuda:0'), 'rec_psnr:': tensor(29.8045, device='cuda:0'), 'rec_nrmse:': tensor(0.2026, device='cuda:0'), 'rec_nrmse2:': tensor(0.0323, device='cuda:0')}
{'pred_ssim:': tensor(0.9511, device='cuda:0'), 'pred_psnr:': tensor(38.9665, device='cuda:0'), 'pred_nrmse:': tensor(0.0706, device='cuda:0'), 'pred_nrmse2:': tensor(0.0113, device='cuda:0')}
